In [ ]:
import torch
import numpy as np

from raytorch.ops.ray_intersect import ray_triangle_intersect_iter, ray_triangle_intersect_single_ray

from pytorch3d.structures import Pointclouds
from pytorch3d.utils import ico_sphere
from pytorch3d.renderer import RayBundle
from pytorch3d.vis.plotly_vis import plot_batch_individually, plot_scene
from pytorch3d.transforms import Scale, Translate


In [ ]:
obj = ico_sphere(level = 1)
translate = Translate(x=10, y=0, z=0)
new_verts = translate.transform_points(obj.verts_padded())
obj = obj.update_padded(new_verts)

vertices = obj.verts_packed()
faces = obj.faces_packed()

print(f"vertex: {vertices.size()}")
print(f"faces: {faces.size()}")

vert_aligned = vertices[faces]

print(f"vert_aligned: {vert_aligned.size()}")

In [ ]:
azimuthal_range = np.arange(-30, 30, 2.5)
polar_range = np.arange(80, 100, 5.0)

light_directions = []
for azimuthal_angle in azimuthal_range:
    for polar_angle in polar_range:
        azimuthal_angle_rad = np.radians(azimuthal_angle)
        polar_angle_rad = np.radians(polar_angle)

        x = np.sin(polar_angle_rad) * np.cos(azimuthal_angle_rad)
        y = np.sin(polar_angle_rad) * np.sin(azimuthal_angle_rad)
        z = np.cos(polar_angle_rad)

        light_directions.append([x, y, z])

light_directions = np.array(light_directions, dtype=np.float32)
origins = np.zeros_like(light_directions, dtype=np.float32)
lengths = np.ones([light_directions.shape[0], 2],
                  dtype=np.float32) * np.array([0.0, 20.0])
xys = np.zeros([light_directions.shape[0], 2], dtype=np.float32)

origins = torch.from_numpy(origins)
light_directions = torch.from_numpy(light_directions)
lengths = torch.from_numpy(lengths)
xys = torch.from_numpy(xys)

print(f"light_directions: {light_directions.shape}")

ray_bundle = RayBundle(origins=origins,
                       lengths=lengths,
                       xys=xys,
                       directions=light_directions)

plot_scene({
    "test": {"ray": ray_bundle,
             "object": obj},
},
    raybundle_ray_line_width=2.0,
    )

In [ ]:
intersection = ray_triangle_intersect_iter(origins,
                            light_directions, 
                            vert_aligned)
print(intersection)

intersection = ray_triangle_intersect_single_ray(origins,
                                           light_directions,
                                           vert_aligned)
print(intersection)
point_cloud = Pointclouds(points=[intersection])

In [ ]:
plot_scene({
    "test": {"ray": ray_bundle,
             "point_cloud": point_cloud},
},
    raybundle_ray_line_width=2.0,
    pointcloud_marker_size=2.0
)